In [15]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.tsp.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class

C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


C:\Users\robin\AppData\Local\Temp\ipykernel_18152\1102566859.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load data

In [16]:
data = []
with File("data/table3_exp.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data.append(dico_value)
df = pd.DataFrame(data)


In [17]:
display(df.head())

,key_dataset,SEED_POINTS,SEED_ASSIGN,NUM_CLUST,NUM_POINTS,IMPR_CLASS,IT_ORDER,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"100,3,6225993,32000,0,32000,1,0",32000,32000,6225993,100,BICL,CURR,4816.909645,368.249957,90,445320,0.011995
1,"100,3,6225993,32001,0,32001,1,0",32001,32001,6225993,100,BICL,CURR,4781.111051,322.833135,101,499748,0.013983
2,"100,3,6225993,32002,0,32002,1,0",32002,32002,6225993,100,BICL,CURR,4954.298247,371.063099,85,420580,0.011485
3,"100,3,6225993,32003,0,32003,1,0",32003,32003,6225993,100,BICL,CURR,4616.307808,320.717186,108,534384,0.014391
4,"100,3,6225993,32004,0,32004,1,0",32004,32004,6225993,100,BICL,CURR,5358.272476,364.514571,87,430476,0.011836


## Verify that we have all of the required results

In [18]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Number of points:

Expected: [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]

Found:

In [19]:
expected = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
num_points = list(df[str(DataCategory.NUM_POINTS)].unique())
num_points.sort()
print(num_points)
check_print(set(num_points) == set(expected))

[20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [20]:
groups_counts = df.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).count().reset_index()
num_points = [20,30,40,50,60,70,80,90,100,110,120,130,140,150,200,250,300,350,400,450,500,600,700,800,900,1000]
expected = {n: 1000 if n<=150 else 100 for n in num_points}
cond = True

for n in expected:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_POINTS)] == expected[n]
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected[n]
check_print(cond)

✅


## Statistics

### Computed statistics

In [21]:
df_table3_src = df[[
    str(DataCategory.NUM_POINTS),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table3 = df_table3_src.groupby([str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
# df_table3 = df_table3.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table3.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table3.reset_index(drop=True, inplace=True)
df_table3_str = df_table3.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table3_str))

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,20,BICL,235.33,15,2730,9.5429e-05
1,30,BICL,257.75,23,10134,2.9913e-04
2,40,BICL,278.19,32,25215,7.9230e-04
3,50,BICL,295.57,42,51563,2.1678e-03
4,60,BICL,310.80,52,91476,3.0751e-03
5,70,BICL,326.12,62,149534,3.8258e-03
6,80,BICL,335.97,72,227828,5.9868e-03
7,90,BICL,348.37,83,330724,8.6032e-03
8,100,BICL,359.66,93,461609,1.2348e-02
9,110,BICL,371.45,103,619532,1.6197e-02


### Reference values

Verification manuelle algorithme par algorithme:

In [22]:
df_table3_ref_src = pd.read_csv("data/table3.csv")
# display(df_table1_ref_src.head())

In [23]:
columns = [c for c in df_table3_ref_src.columns if "percent" not in c and "NUM_POINTS" not in c]
algorithms = set([c.split("-")[1] for c in columns])
new_columns = set([c.split("-")[0] for c in columns])
data = []
for n in df_table3_ref_src[str(DataCategory.NUM_POINTS)].unique():
    n = int(n)
    for algo in algorithms:
        dico = {
            str(DataCategory.NUM_POINTS): n,
            str(DataCategory.IMPR_CLASS): algo,
        }
        for col in new_columns:
            dico[col] = df_table3_ref_src.query(f"{str(DataCategory.NUM_POINTS)} == {n}")[f"{col}-{algo}"].to_numpy()[0]
        data.append(dico)
df_table3_ref = pd.DataFrame(data)
df_table3_ref.sort_values(by=[str(DataCategory.NUM_POINTS),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table3_ref.reset_index(drop=True,inplace=True)
df_table3_ref = df_table3_ref[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","CPU_time"]]
pd.set_option('display.max_rows', 1000)
display(df_table3_ref)

,NUM_POINTS,IMPR_CLASS,final_cost,num_iter,CPU_time
0,20,BINL,229.83,14,0.00
1,30,BINL,255.36,23,0.00
2,40,BINL,276.74,33,0.01
3,50,BINL,293.80,42,0.03
4,60,BINL,309.20,52,0.06
5,70,BINL,324.51,62,0.11
6,80,BINL,337.56,72,0.16
7,90,BINL,347.36,82,0.23
8,100,BINL,359.88,93,0.33
9,110,BINL,373.09,103,0.44


✅ok, same values ❌, different values

|Algorithme \ Metrique| final_cost | num_iter | duration (s)|
|:---|:---:|:---:|:---:|
|BINL|✅|✅|❌
|FICL|✅|✅|❌
|FINL|✅|✅|❌

In [24]:
computed = df_table3.copy()
reference = df_table3_ref.copy()
reference.rename(columns={"CPU_time":"duration"}, inplace=True)

computed = computed[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","duration"]]

diff = []
for n in reference[str(DataCategory.NUM_POINTS)].unique():
    for impr_class in reference[str(DataCategory.IMPR_CLASS)].unique():
        df_sel_ref = reference.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        df_sel_comp = computed.query(f"{str(DataCategory.NUM_POINTS)} == {n} and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        diff.append(
            {
                str(DataCategory.NUM_POINTS):n,
                str(DataCategory.IMPR_CLASS):impr_class, 
                str(DataCategory.final_cost)+"_ref":df_sel_ref.iloc[0]["final_cost"], 
                str(DataCategory.final_cost)+"_comp":df_sel_comp.iloc[0]["final_cost"], 
                str(DataCategory.final_cost)+"_diff":df_sel_comp.iloc[0]["final_cost"]-df_sel_ref.iloc[0]["final_cost"], 
                str(DataCategory.num_iter)+"_ref":df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_comp":df_sel_comp.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_diff":df_sel_comp.iloc[0]["num_iter"]-df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.duration)+"_ref":df_sel_ref.iloc[0]["duration"], 
                str(DataCategory.duration)+"_comp":df_sel_comp.iloc[0]["duration"],
                str(DataCategory.duration)+"_diff":df_sel_comp.iloc[0]["duration"]-df_sel_ref.iloc[0]["duration"]
            }
        )
df_diff = pd.DataFrame(diff)
df_diff_str = df_diff.to_html(formatters={
    str(DataCategory.NUM_POINTS): '{:.0f}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost)+"_ref": '{:.2f}'.format,
    str(DataCategory.final_cost)+"_comp": '{:.2f}'.format,
    str(DataCategory.final_cost)+"_diff": '{:.2f}'.format,
    str(DataCategory.num_iter)+"_ref": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_comp": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_diff": '{:.0f}'.format,
    str(DataCategory.duration)+"_ref": '{:.4e}'.format,
    str(DataCategory.duration)+"_comp": '{:.4e}'.format,
    str(DataCategory.duration)+"_diff": '{:.4e}'.format,
})
display(HTML(df_diff_str))
for col in [str(DataCategory.final_cost),str(DataCategory.num_iter),str(DataCategory.duration)]:
    df_diff[f"{col}_diff_abs"] = df_diff[f"{col}_diff"].abs()
    description = df_diff[f"{col}_diff_abs"].describe().to_dict()
    print(f"{col}_diff_abs: {description['min']:.2e}, {description['25%']:.2e}, {description['50%']:.2e}, {description['mean']:.2e}, {description['std']:.2e}, {description['75%']:.2e}, {description['max']:.2e}")


,NUM_POINTS,IMPR_CLASS,final_cost_ref,final_cost_comp,final_cost_diff,num_iter_ref,num_iter_comp,num_iter_diff,duration_ref,duration_comp,duration_diff
0,20,BINL,229.83,234.06,4.23,14,14,0,0.0000e+00,1.5194e-04,1.5194e-04
1,20,FICL,236.90,241.10,4.20,34,35,1,0.0000e+00,4.5154e-05,4.5154e-05
2,20,FINL,224.51,234.47,9.96,23,21,-2,0.0000e+00,7.0589e-05,7.0589e-05
3,30,BINL,255.36,256.43,1.07,23,23,0,0.0000e+00,6.2135e-04,6.2135e-04
4,30,FICL,263.12,266.32,3.20,61,63,2,0.0000e+00,2.2747e-04,2.2747e-04
5,30,FINL,242.85,243.82,0.97,38,34,-4,0.0000e+00,1.7798e-04,1.7798e-04
6,40,BINL,276.74,277.34,0.60,33,33,-0,1.0000e-02,1.1663e-03,-8.8337e-03
7,40,FICL,284.92,286.53,1.61,91,94,3,0.0000e+00,4.1155e-04,4.1155e-04
8,40,FINL,256.77,256.07,-0.70,54,48,-6,0.0000e+00,3.4737e-04,3.4737e-04
9,50,BINL,293.80,296.36,2.56,42,42,-0,3.0000e-02,2.0378e-03,-2.7962e-02


final_cost_diff_abs: 8.05e-04, 1.10e+00, 2.13e+00, 4.00e+00, 5.18e+00, 4.18e+00, 2.43e+01
num_iter_diff_abs: 4.00e-02, 1.41e+00, 9.93e+00, 3.26e+01, 4.98e+01, 3.65e+01, 2.10e+02
duration_diff_abs: 4.52e-05, 2.82e-02, 2.41e-01, 1.80e+01, 6.63e+01, 2.14e+00, 4.31e+02


## Export data for visualization

In [ ]:
import json
computed = df_table3_src.copy()
reference = df_table3_ref.copy()
reference.rename(columns={"CPU_time":"duration"}, inplace=True)

computed = computed[["NUM_POINTS","IMPR_CLASS","final_cost","num_iter","duration"]]
computed["source"] = "computed"
reference["source"] = "reference"
new_df = pd.concat([computed,reference])
display(new_df)
def q1(x):
    return x.quantile(0.25)

def median(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)
f = ["mean","std","min",q1,median,q3,"max"]
df_avg = new_df.groupby(["NUM_POINTS","IMPR_CLASS","source"]).agg(f)
df_avg.reset_index(inplace=True)
df_avg.fillna(-1,inplace=True)
Lout = []
for elem in df_avg.to_dict(orient="records"):
    dico = {}
    for k,v in elem.items():
        if k[1] == '':
            dico[k[0]] = v
        else:
            if k[0] not in dico:
                dico[k[0]] = {}
            dico[k[0]][k[1]] = v
    dico["INIT_CHOICE"] = "random"
    Lout.append(dico)
dico_filtered = {}
for n in new_df["NUM_POINTS"].unique():
    for impr_class in new_df["IMPR_CLASS"].unique():
        for source in new_df["source"].unique():
            tmp = new_df.query(f"NUM_POINTS == {n} and IMPR_CLASS == '{impr_class}' and source == '{source}'").copy()
            tmp_dico = tmp.to_dict(orient="records")
            for i in range(len(tmp_dico)):
                tmp_dico[i]["INIT_CHOICE"] = "random"
            if len(tmp_dico) == 0:
                continue
            dico_filtered[f"{n}_{impr_class}_{source}"] = tmp_dico
            
dico_out = {"grouped":Lout,"filtered":dico_filtered}

with open("data/table3_visualize.json","w") as f:
    json.dump(dico_out,f,indent=2)

In [25]:
from itertools import product
data = {
    ('NUM_POINTS',''):[],
    ('Obj. function values','BICL'):[],
    ('Obj. function values','BINL'):[],
    ('Obj. function values','FICL'):[],
    ('Obj. function values','FINL'):[],
    ('CPU times','BICL'):[],
    ('CPU times','BINL'):[],
    ('CPU times','FICL'):[],
    ('CPU times','FINL'):[],
    ('# of iterations','BICL'):[],
    ('# of iterations','BINL'):[],
    ('# of iterations','FICL'):[],
    ('# of iterations','FINL'):[],
}
nPts = df_table3["NUM_POINTS"].unique()
imprClass = df_table3["IMPR_CLASS"].unique()
attribute = [["final_cost",'Obj. function values'],["duration",'CPU times'],["num_iter",'# of iterations']]
for npt,impr in product(nPts,imprClass):
    tmp_df = df_table3.query(f"NUM_POINTS == {npt} and IMPR_CLASS == '{impr}'")
    try:
        dico = tmp_df.to_dict(orient="records")[0]
    except Exception:
        continue
    if npt not in data[('NUM_POINTS','')]:
        data[('NUM_POINTS','')].append(npt)
    for name,label in attribute:
        data[(label,impr)].append(float(f'{dico[name]:.2f}'))
pd.set_option('display.max_rows', None)
print([len(data[k]) for k in data.keys()])
df_as_paper = pd.DataFrame(data)
df_as_paper[('% impr.','FICL')] = ((df_as_paper[('Obj. function values','BINL')]-df_as_paper[('Obj. function values','FICL')])/df_as_paper[('Obj. function values','BINL')]*100).round(2)
df_as_paper[('% impr.','FINL')] = ((df_as_paper[('Obj. function values','BINL')]-df_as_paper[('Obj. function values','FINL')])/df_as_paper[('Obj. function values','BINL')]*100).round(2)
cols = df_as_paper.columns.tolist()
df_as_paper = df_as_paper[cols[:4]+cols[-2:]+cols[5:-2]]
display(df_as_paper)
pd.set_option('display.max_rows', 20)

[26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]


NUM_POINTS Obj. function values                 % impr.        CPU times  \
                              BICL    BINL    FICL    FICL   FINL      BICL   
0          20               235.33  234.06  241.10   -3.01  -0.18      0.00   
1          30               257.75  256.43  266.32   -3.86   4.92      0.00   
2          40               278.19  277.34  286.53   -3.31   7.67      0.00   
3          50               295.57  296.36  304.60   -2.78  10.44      0.00   
4          60               310.80  310.45  319.33   -2.86  11.22      0.00   
5          70               326.12  324.72  333.25   -2.63  13.64      0.00   
6          80               335.97  336.07  347.78   -3.48  14.68      0.01   
7          90               348.37  350.16  360.33   -2.90  16.26      0.01   
8         100               359.66  360.14  372.45   -3.42  16.85      0.01   
9         110               371.45  371.66  382.65   -2.96  18.16      0.02   
10        120               381.57  380.06  394.90   -3.90  18.75      0.02   
11        130               393.51  392.07  405.79   -3.50  20.08      0.03   
12        140               403.79  400.75  413.25   -3.12  20.67      0.03   
13        150               411.44  411.75  424.76   -3.16  21.94      0.04   
14        200               451.71  450.58  466.96   -3.64  25.10      0.11   
15        250               493.01  494.83  510.37   -3.14  27.01      0.21   
16        300               529.02  526.68  543.52   -3.20  29.16      0.38   
17        350               560.38  560.32  581.17   -3.72  32.58      0.61   
18        400               589.46  590.03  611.83   -3.69  32.49      0.94   
19        450               619.84  617.45  636.05   -3.01  33.83      1.43   
20        500               645.92  646.33  667.36   -3.25  35.19      1.97   
21        600               693.85  697.81  721.57   -3.40  37.53      3.49   
22        700               740.63  739.50  771.56   -4.34  38.92      5.53   
23        800               787.55  787.02  807.15   -2.56  40.58      8.78   
24        900               827.94  826.89  849.62   -2.75  41.96     12.36   
25       1000               864.02  864.63  892.25   -3.19  42.71     19.55   

                     # of iterations                             
    BINL  FICL  FINL            BICL     BINL     FICL     FINL  
0   0.00  0.00  0.00           14.52    14.35    35.43    20.72  
1   0.00  0.00  0.00           23.41    23.28    62.96    34.18  
2   0.00  0.00  0.00           32.41    32.56    94.05    48.26  
3   0.00  0.00  0.00           42.16    41.78   126.97    62.68  
4   0.00  0.00  0.00           51.74    51.82   160.60    77.81  
5   0.00  0.00  0.00           61.97    61.91   196.30    92.21  
6   0.01  0.00  0.00           72.14    72.12   232.86   107.92  
7   0.01  0.00  0.00           82.62    82.28   271.08   123.14  
8   0.01  0.00  0.00           93.29    92.56   310.67   138.93  
9   0.01  0.00  0.00          103.38   103.04   348.96   154.29  
10  0.02  0.01  0.00          114.13   114.21   390.31   169.84  
11  0.02  0.01  0.00          124.64   124.54   430.82   186.14  
12  0.03  0.01  0.00          134.99   135.30   472.62   201.13  
13  0.03  0.01  0.00          146.44   146.26   513.90   217.47  
14  0.08  0.02  0.01          204.11   201.53   733.18   299.12  
15  0.15  0.04  0.01          258.14   256.72   961.52   380.35  
16  0.25  0.06  0.01          317.97   316.30  1193.07   460.98  
17  0.40  0.09  0.02          373.36   372.60  1423.62   545.12  
18  0.58  0.13  0.03          433.19   430.16  1680.77   629.62  
19  0.86  0.18  0.03          490.63   489.73  1926.99   714.71  
20  1.19  0.25  0.04          552.75   552.81  2187.71   798.06  
21  1.95  0.39  0.06          670.40   669.64  2707.63   966.90  
22  3.06  0.56  0.08          792.06   793.38  3255.25  1139.19  
23  4.69  0.84  0.11          913.42   911.51  3807.26  1315.37  
24  6.96  1.21  0.14         1035.25  1035.42  4381.54  1490.61  
25  9.59